# 0. Install Dependencies

In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [35]:
import gym 
import random

In [36]:
env = gym.make('CartPole-v1',render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [37]:
actions

2

In [38]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info,_ = env.step(action)
        print(env.step(action))
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

(array([ 0.02721376, -0.37880325,  0.01586404,  0.5594002 ], dtype=float32), 1.0, False, False, {})
(array([ 0.00815481, -0.7696241 ,  0.04419282,  1.1581035 ], dtype=float32), 1.0, False, False, {})
(array([-0.01873977, -0.38095975,  0.08494686,  0.608828  ], dtype=float32), 1.0, False, False, {})
(array([-0.03790217, -0.77344984,  0.11566365,  1.248568  ], dtype=float32), 1.0, False, False, {})
(array([-0.07276814, -1.1663395 ,  0.17213756,  1.9081641 ], dtype=float32), 1.0, False, False, {})
(array([-0.12335196, -1.5592449 ,  0.25527954,  2.598408  ], dtype=float32), 0.0, True, False, {})
Episode:1 Score:6.0
(array([ 0.03188365, -0.34964645,  0.0417469 ,  0.5707575 ], dtype=float32), 1.0, False, False, {})
(array([ 0.01398416, -0.7411309 ,  0.07068794,  1.1852058 ], dtype=float32), 1.0, False, False, {})
(array([-0.01177833, -0.353266  ,  0.11270189,  0.6539043 ], dtype=float32), 1.0, False, False, {})
(array([-0.02204123,  0.03325534,  0.13375454,  0.14819738], dtype=float32), 1.0,

# 2. Create a Deep Learning Model with Keras

In [39]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [40]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [41]:
model = build_model(states, actions)

In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [43]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [44]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [45]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_2_input to have shape (1, 4) but got array with shape (1, 2)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)